# Sector Visualization

In this example, we will parse the database and visualize each sector's averaage times.

## Step 0: Global variables

In [1]:
database_path = 'bike_data.db'

## Step 1: Import required libraries and the database

In [2]:
import pandas as pd
from sqlalchemy import create_engine
from ipyleaflet import Map, Marker, Polygon, Polyline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
import math
import plotly.subplots
import plotly.graph_objects

# SQLAlchemy connectable 
conn = create_engine( 'sqlite:///' + database_path ).connect()

## Step 2: Import Sector Data

In [3]:
sector_list = pd.read_sql_table( 'sector_list',conn)
centroid = [0, 0]
num_points = 0

sector_polygons = {}
sector_polygons['all'] = {'centroid': (0,0),
                          'polygon': []}
counter = 0.0
for sector in sector_list['sector_id']:
    
    sector_polygons[sector] = {'centroid': (0, 0),
                               'polygon': []}
    sector_points = pd.read_sql_table( sector, conn )
    
    centroid = [centroid[0] + sector_points['latitude'].mean()  * len(sector_points),
                centroid[1] + sector_points['longitude'].mean() * len(sector_points)]
    num_points += len( sector_points )
    sector_polygons[sector]['centroid'] =  [sector_points['latitude'].mean(),
                                            sector_points['longitude'].mean()]
    
    color_code = matplotlib.colors.rgb2hex(plt.get_cmap('hsv')( counter / len(sector_list) ))
    polygon = [ ( row['latitude'], row['longitude'] ) for index, row in sector_points.iterrows() ]
    sector_polygons['all']['polygon'].append( Polygon( locations=polygon,
                                                        color=color_code,
                                                        fill_color=color_code ) )
    sector_polygons[sector]['polygon'].append( Polygon( locations=polygon,
                                                        color=color_code,
                                                        fill_color=color_code ) )
    counter += 1.0

Here is a handy visualization of the designated sectors.

In [4]:
centroid_pt = (centroid[0]/num_points, 
               centroid[1]/num_points)

#  Build Map Visualization    
sector_poly_map = Map( center=centroid_pt, zoom=11)
for polygon in sector_polygons['all']['polygon']:
    sector_poly_map.add_layer(polygon)
    
sector_poly_map.layout.height="550px"    
sector_poly_map

Map(center=[39.68022141177373, -104.91524623127385], controls=(ZoomControl(options=['position', 'zoom_in_text'…

For each sector, we need to construct tracks

In [5]:
def Build_Segment( conn, point_list, dataset_id, sector_id ):
    
    #  Get a list of all points with that sector-id
    sql_query = 'SELECT * FROM point_list WHERE (sector_id = \'{}\') and (dataset_id = {})'.format( sector_id, dataset_id )
    points_in_sector = pd.read_sql_query( sql_query, conn )
    print(points_in_sector)
    
    return points_in_sector

Construct the tracks

In [6]:
#  For each segment, we need to create a track for each dataset
dataset_ids = pd.read_sql_query('SELECT DISTINCT dataset_id FROM point_list', conn)

#  Load the entire list of points into one dataframe for quicker access
point_list = pd.read_sql_table( 'point_list', conn )

# List of Polylines to view
sector_objects = {}
sector_objects['all'] = {'route': {},
                         'elevation_plot': []}
for x in range( 0, len(dataset_ids['dataset_id'])):
        sector_objects['all']['elevation_plot'].append( [[],[]] )
    
for x in sector_list['sector_id']:
    sector_objects[x] = {'route': {},
                         'elevation_plot': []}
    
global_polylines = {}
for dataset_id in dataset_ids['dataset_id']:
    global_polylines[dataset_id] = []

    
#  Iterate over points    
for sector_id in sector_list['sector_id']:
    
    for x in range( 0, len(dataset_ids['dataset_id'])):
        sector_objects[sector_id]['elevation_plot'].append( [[],[]] )
    
    #  For each dataset
    for dataset_id in dataset_ids['dataset_id']:
        
        #  Create a full track of the segment
        sql_query = 'SELECT * FROM point_list WHERE (sector_id = \'{}\') and (dataset_id = {}) ORDER BY timestamp'.format( sector_id, dataset_id )
        points_in_sector = pd.read_sql_query( sql_query, conn )
        
        #  from the point list, we can compute time spent in sector
        points_in_sector['timestamp'] =  pd.to_datetime(points_in_sector['timestamp'])
        time_in_sector = points_in_sector['timestamp'].max() - points_in_sector['timestamp'].min()
        
        #  We can build a linked-list of points
        distance_sector = 0
        sector_polyline = []
        sector_objects[sector_id]['elevation_plot']
        for point in points_in_sector.itertuples( index=False ):
            
            #  Update the Polylines
            global_polylines[dataset_id].append( [ point[2], point[1] ] )
            sector_polyline.append( [point[2], point[1] ] )
            
            #  Elevation Information
            sector_objects['all']['elevation_plot'][dataset_id][0].append( pd.to_datetime(point[4]) - dataset_start )
            sector_objects['all']['elevation_plot'][dataset_id][1].append( point.elevation )
            sector_objects[sector_id]['elevation_plot'][dataset_id][0].append( pd.to_datetime(point[4]) - sector_start )
            sector_objects[sector_id]['elevation_plot'][dataset_id][1].append( point.elevation )
            
        sector_objects[sector_id]['route'][dataset_id] = Polyline( locations=sector_polyline,
                                                                   color='blue',
                                                                   fill=False )
            
        print('Sector: {}, Dataset: {}, Distance: {}'.format( sector_id, dataset_id, distance_sector ) )
        #  We can also find the distance run through sector
        #distance_in_sector
        
for dataset_id in dataset_ids['dataset_id']:
    sector_objects['all']['route'][dataset_id] = Polyline( locations=global_polylines[dataset_id], color='blue', fill=False )


OperationalError: (sqlite3.OperationalError) no such column: dataset_id
[SQL: SELECT DISTINCT dataset_id FROM point_list]
(Background on this error at: http://sqlalche.me/e/13/e3q8)

## Plotting Data 

It would be nice to visualize our map alongside some useful plots.

### Control Widget
Let's make a widget to control which sector we want to view.

In [ ]:
import ipywidgets as widgets

sector_names = [x for x in sector_list['sector_id']]
sector_names = [ 'all' ] + sector_names
sector_select_widget = widgets.Dropdown( options=sector_names,
                                         value='all',
                                         description='Sector:',
                                         disabled=False )
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        
        #  Remove Routes
        print('Old Sector: {}'.format(change['old']))
        for dataset_id in sector_objects[change['old']]['route']:
            sector_map.remove_layer( sector_objects[change['old']]['route'][dataset_id])
        
        #  Remove Widgets
        for polygon in sector_polygons[change['old']]['polygon']:
            sector_map.remove_layer(polygon)
        
        #  Add Routes
        print('New Sector: {}'.format(change['new']))
        for dataset_id in sector_objects[change['new']]['route']:
            sector_map.add_layer( sector_objects[change['new']]['route'][dataset_id])
            
        for polygon in sector_polygons[change['new']]['polygon']:
            sector_map.add_layer(polygon)

sector_select_widget.observe(on_change)

### Map Widget


In [ ]:
#  Build Map Visualization    
sector_map = Map( center=centroid_pt, zoom=11)
for dataset_id in sector_objects[sector_select_widget.value]['route']:
    sector_map.add_layer( sector_objects[sector_select_widget.value]['route'][dataset_id] )
    
for polygon in sector_polygons[sector_select_widget.value]['polygon']:
            sector_map.add_layer(polygon)    
#sector_map.layout.height="400px"

### Plot Data

In [ ]:
elevation_profile_fig = plotly.graph_objects.Figure()
elevation_profile_fig.update_layout( title={ 'text': "Elevation Profile" } )

#  For each Dataset,
for dataset_id in dataset_ids['dataset_id']:
    elevation_profile_fig.add_trace( plotly.graph_objects.Scatter( x=sector_objects[sector_select_widget.value]['elevation_plot'][dataset_id][0], 
                                                            y=sector_objects[sector_select_widget.value]['elevation_plot'][dataset_id][1],
                                                            mode='lines',
                                                            name='lines'))

## Tying it all together...

In [ ]:
sector_select_widget

In [ ]:
sector_map

In [ ]:
elevation_profile_fig.show( config = dict({'scrollZoom': True}) )

## References

* Vincenty Formula Code: https://nathanrooy.github.io/posts/2016-12-18/vincenty-formula-with-python/